## **Movie Recommender Sysytem**

In [1]:
import pandas as pd

In [ ]:
rating = pd.read_csv(r"D:\Python-SQL-Repository\Dataset\Movies\movies.csv")
movie = pd.read_csv(r"D:\Python-SQL-Repository\Dataset\Movies\ratings.csv")

In [8]:
rating.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [9]:
movie.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [10]:
data = pd.merge(rating, movie, on='movieId')
data.head()

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [12]:
data[data['movieId'] == 2]

,userId,movieId,rating,timestamp,title,genres
560,6,2,4.0,845553522,Jumanji (1995),Adventure|Children|Fantasy
1026,8,2,4.0,839463806,Jumanji (1995),Adventure|Children|Fantasy
1773,18,2,3.0,1455617462,Jumanji (1995),Adventure|Children|Fantasy
2275,19,2,3.0,965704331,Jumanji (1995),Adventure|Children|Fantasy
2977,20,2,3.0,1054038313,Jumanji (1995),Adventure|Children|Fantasy
...,...,...,...,...,...,...
95102,600,2,4.0,1237764627,Jumanji (1995),Adventure|Children|Fantasy
95965,602,2,4.0,840875851,Jumanji (1995),Adventure|Children|Fantasy
97044,604,2,5.0,832080293,Jumanji (1995),Adventure|Children|Fantasy
97144,605,2,3.5,1277176522,Jumanji (1995),Adventure|Children|Fantasy


In [15]:
data.groupby('title')['rating'].mean().sort_values(ascending=False).head()

title
Karlson Returns (1970)                                                         5.0
Zeitgeist: Moving Forward (2011)                                               5.0
Dream of Light (a.k.a. Quince Tree Sun, The) (Sol del membrillo, El) (1992)    5.0
Dragons: Gift of the Night Fury (2011)                                         5.0
12 Angry Men (1997)                                                            5.0
Name: rating, dtype: float64

In [16]:
data.groupby('title')['rating'].mean().sort_values(ascending=False).tail()

title
The Pumaman (1980)                      0.5
3 dev adam (Three Giant Men) (1973)     0.5
3 Ninjas Knuckle Up (1995)              0.5
The Gracefield Incident (2015)          0.5
Bloodsport: The Dark Kumite (1999)      0.5
Name: rating, dtype: float64

In [17]:
data.groupby('title')['userId'].count().sort_values(ascending=False).head()

title
Forrest Gump (1994)                 329
Shawshank Redemption, The (1994)    317
Pulp Fiction (1994)                 307
Silence of the Lambs, The (1991)    279
Matrix, The (1999)                  278
Name: userId, dtype: int64

In [21]:
user_freq = data[['userId', 'movieId']].groupby('userId').count().reset_index().rename({'movieId':'n_movies'}).head()
user_freq

,userId,movieId
0,1,232
1,2,29
2,3,39
3,4,216
4,5,44


In [25]:
movie.movieId.count()

np.int64(9742)

In [23]:
data['genres']

0         Adventure|Animation|Children|Comedy|Fantasy
1                                      Comedy|Romance
2                               Action|Crime|Thriller
3                                    Mystery|Thriller
4                              Crime|Mystery|Thriller
                             ...                     
100831                          Drama|Horror|Thriller
100832                          Action|Crime|Thriller
100833                                         Horror
100834                                  Action|Sci-Fi
100835                    Action|Crime|Drama|Thriller
Name: genres, Length: 100836, dtype: object

In [26]:
unique_geners = set()

for genres in data['genres']:
    genre_list = genres.split('|')
    for genre in genre_list:
        unique_geners.add(genre)
    

In [27]:
all_genres = sorted(list(unique_geners))
all_genres

['(no genres listed)',
 'Action',
 'Adventure',
 'Animation',
 'Children',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'IMAX',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western']

In [28]:
def genre_to_vector(genres, all_genres):
    vector = [0] * len(all_genres)
    genre_list = genres.split('|')
    for genre in genre_list:
        if genre in all_genres:
            index = all_genres.index(genre)
        vector[index] = 1
    return vector

In [29]:
genre_to_vector(data['genres'][1], all_genres)

[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]

In [31]:
# creat MoviDict

movieDict = {}
for index, row in data.iterrows():
    movieId = row['movieId']
    title = row['title']
    genre_vector = genre_to_vector(row['genres'], all_genres)
    rating = row['rating']
    movieDict[movieId] = (title, genre_vector, rating)


In [32]:
movieDict[1]

('Toy Story (1995)',
 [0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 5.0)

In [34]:
from scipy.spatial.distance import cosine

    

In [35]:
def compute_distence(movie_a, movie_b):
    genre_distance = cosine(movie_a[1], movie_b[1])
    rating_distance = abs(movie_a[2] - movie_b[2])
    total_distance = genre_distance+rating_distance
    return total_distance, genre_distance, rating_distance

In [52]:
import operator
def get_neighbors(movie_id, k):
    distances = []
    for movie in movieDict:
        if movie != movie_id:
            dist = compute_distence(movieDict[movie_id], movieDict[movie])
            distances.append((movie, dist))

    distances.sort(key=operator.itemgetter(1))

    neighbors = []
    for i in range(min(k, len(distances))):
        neighbors.append(distances[i][0])
    return neighbors

In [53]:
# Compute the average rating of the K nearest neighbors
movie_id = int(input("Enter the movie ID: "))

neighbors = get_neighbors(movie_id, k=5)
avg_rating = 0
for neighbor in neighbors:
    avg_rating += movieDict[neighbor][2]
    print(f"{movieDict[neighbor][0]} {movieDict[neighbor][2]}")

if neighbors:
    avg_rating /= len(neighbors)
    print(f"Average Rating of Nearest Neighbors: {avg_rating}")
else:
    print("No neighbors found.")

Enter the movie ID:  1


Toy Story 2 (1999) 5.0
Asterix and the Vikings (Astérix et les Vikings) (2006) 5.0
Moana (2016) 5.0
Toy Story 3 (2010) 5.0
Penguins of Madagascar (2014) 5.0
Average Rating of Nearest Neighbors: 5.0
